In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import numpy as np
import pandas as pd

Mounted at /content/gdrive


In [0]:
def matrix_factorization(R, P, Q, K, miu,bu,bi , steps=2, alpha=0.02, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij =R[i][j] - miu  - bu[i] - bi[j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

In [0]:
def itembias(items,npMovielens,miu):
  bi=[]
  for x in range(len(items)):
    itemId = items[x]
    ratings = npMovielens[npMovielens[:,1] == itemId, 2] # select all ratings by itemid
    averageRating=ratings.mean()
    itemBias = ratings.mean() - miu
    bi.append(itemBias)
  return bi
def userbias(users,npMovielens,miu):
  bu=[]
  for x in range(len(users)):
    userId = users[x]
    ratings = npMovielens[npMovielens[:,0] == userId, 2] # select all ratings by userid
    averageRating=ratings.mean()
    userBias = ratings.mean() - miu
    bu.append(userBias)
    verageRating = ratings.mean()
    #print(userId, averageRating,userBias)
    
  return bu

In [0]:
if __name__ == "__main__":
    header = ['user_id', 'item_id', 'rating', 'timestamp']
    drivePrefix = '/content/gdrive/My Drive/MovieLens/'
    driveSuffix = 'ratings.csv'
    df = pd.read_csv(drivePrefix + driveSuffix, sep=',', names =header, skiprows=1)
    npMovielens = df.to_numpy()
    users = np.unique(npMovielens[:,0])
    items = np.unique(npMovielens[:,1])
    n_users = len(users)
    n_items = len(items)
    # print('Number of users = ' + str(n_users) + ' | number of movies = ' + str(n_items))
    # print(df)

In [0]:
    #5-fold for test
    test1 = df.sample(frac=0.2,random_state=200) 
    drop1=df.drop(test1.index)
    test_2 = drop1.sample(frac=0.2499969009,random_state=200)
    drop2=drop1.drop(test_2.index)
    test_3 = drop2.sample(frac=0.3333278239,random_state=200)
    drop3= drop2.drop(test_3.index)
    test_4 = drop3.sample(frac=0.4999876038,random_state=200)
    test = drop3.drop(test_4.index)   
    test_rating=test
    users_unique=test.user_id.unique()

In [6]:
    test['rating']=0.0
    # pd.DataFrame(test)
    training=df.drop(test.index)
    frame=[training,test]
    train=pd.concat(frame)
    pd.DataFrame(train)

,user_id,item_id,rating,timestamp
1,1,3,4.0,964981247
5,1,70,3.0,964982400
6,1,101,5.0,964980868
8,1,151,5.0,964984041
9,1,157,5.0,964984100
...,...,...,...,...
100821,610,160527,0.0,1479544998
100822,610,160571,0.0,1493848537
100828,610,163981,0.0,1493850155
100832,610,168248,0.0,1493850091


In [7]:
    matrix_rating= df.pivot('user_id','item_id','rating').values
    matrix_rating=np.nan_to_num(matrix_rating)
    #Test = np.array(matrix_rating)
    pd.DataFrame(matrix_rating)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9684,9685,9686,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698,9699,9700,9701,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.5,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.5,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
606,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,2.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
    N = len(matrix_rating)
    M = len(matrix_rating[0])
    Rec=[]
    K = 4
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)
    ratings = npMovielens[:, 2] # select all ratings in the training set
    miu = ratings.mean()
    # miu=globalaverage(matrix_rating)
    bu = userbias(users,npMovielens,miu)
    bi = itembias(items,npMovielens,miu)
    print(bu)

[0.8648223267278654, 0.4467188784520033, -1.0656595477195263, 0.053998571938593365, 0.13480665274667425, -0.00792641036855457, -0.27129382572222527, 0.07291110148942082, -0.24068741839957086, -0.22298555504553352, 0.279693016383038, 0.889068016383038, 0.14360430670561852, -0.10572365028362851, -0.05340883546881381, 0.22293281230140538, 0.7079668259068477, 0.23051472953044838, -0.8941601130621968, 0.08935210729212884, -0.24083463598716515, -0.9301284121883904, 0.14720334696155035, 0.14844301638303792, 1.3061353240753455, -0.26346174552172386, 0.04659116453118628, -0.4813815450204708, 0.6404183250250135, 1.2337371340300969, 0.41844301638303794, 0.2533449771673517, 0.28690455484457633, -0.08295233245417144, 0.5853995381221688, -0.8682236502836287, 0.6413001592401812, -0.28360826566824393, 0.498443016383038, 0.26543330764517403, -0.2481007624188054, 0.06435210729212892, 1.0510745953304061, -0.14739031695029547, 0.37438286600709825, 0.498443016383038, -0.4479855550455336, 0.5287460466860683

In [9]:
    nP, nQ = matrix_factorization(matrix_rating, P, Q, K,miu,bu,bi)
    Rec = np.dot(nP, nQ.T)+miu+bi
    print(Rec)
    for i in range(len(bu)):
      Reco = Rec+bu[i]
    print(Reco)

[[3.97154952 3.44289546 3.28624432 ... 3.48469791 3.55063912 4.06451831]
 [3.8373804  3.21347503 3.12098254 ... 3.16465314 3.14698553 3.89756759]
 [3.90092881 3.38749599 3.20666943 ... 3.73881881 4.10591846 4.51063419]
 ...
 [3.9509704  3.53714644 3.27618311 ... 4.08603299 4.75640182 4.68724673]
 [3.94966817 3.49591602 3.26328512 ... 3.91969012 4.49276979 4.52314572]
 [3.91722945 3.39299079 3.23732495 ... 3.53454867 3.60721346 4.20782239]]
[[4.1585486  3.62989454 3.47324341 ... 3.671697   3.7376382  4.25151739]
 [4.02437948 3.40047412 3.30798162 ... 3.35165222 3.33398461 4.08456667]
 [4.08792789 3.57449508 3.39366851 ... 3.92581789 4.29291755 4.69763327]
 ...
 [4.13796949 3.72414552 3.46318219 ... 4.27303207 4.94340091 4.87424582]
 [4.13666726 3.68291511 3.45028421 ... 4.1066892  4.67976887 4.7101448 ]
 [4.10422853 3.57998987 3.42432404 ... 3.72154775 3.79421254 4.39482148]]


In [10]:
#find item recommendation
movies = pd.read_csv("/content/gdrive/My Drive/MovieLens/movies.csv") 
pd.DataFrame(movies)



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [0]:
npTest=test_rating.to_numpy()
users = np.unique(npTest[:,0])
n_users=len(users)
items=np.unique(npTest[:,1])
All_test=[]
for x in range(len(users)):
  userId = users[x]
  userId = int(userId)-1
  ratings = Reco[userId]
  R_sort= np.argsort(-1*ratings)[:10]
  top20=[]
  for j in range(len(R_sort)):
    movie_id= R_sort[j]+1 
    item_rec = movies.iloc[movie_id]['title']
    top20.append(item_rec)
  All_test.append(top20)

In [12]:
data = {'user_id': users,'recommendation': All_test}
Hasil_RecCF = pd.DataFrame(data)
pd.DataFrame(Hasil_RecCF)

,user_id,recommendation
0,1.0,"[Extreme Ops (2002), Blood Work (2002), Beauti..."
1,2.0,"[Big Green, The (1995), All That Jazz (1979), ..."
2,3.0,"[Faces of Death (1978), Amateur (1994), Meet t..."
3,4.0,"[Generation Iron 2, Into the Abyss (2011), Big..."
4,5.0,"[Four Days in September (O Que É Isso, Companh..."
...,...,...
605,606.0,"[Sky High (2003), Kung Fury (2015), Tangled Ev..."
606,607.0,"[Independence Day: Resurgence (2016), Extreme ..."
607,608.0,[Italian for Beginners (Italiensk for begynder...
608,609.0,[Italian for Beginners (Italiensk for begynder...


## **Content Based using clustering movie by genre**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

In [0]:
cv_ge=CountVectorizer()
genres = cv_ge.fit_transform(movies["genres"])
genres_list = ["genres_"+ i for i in cv_ge.get_feature_names()]
cluster_data = np.hstack([genres.todense()*2])
mod = KMeans(n_clusters=50)
category = mod.fit_predict(cluster_data)

In [15]:
print(genres_list)

['genres_action', 'genres_adventure', 'genres_animation', 'genres_children', 'genres_comedy', 'genres_crime', 'genres_documentary', 'genres_drama', 'genres_fantasy', 'genres_fi', 'genres_film', 'genres_genres', 'genres_horror', 'genres_imax', 'genres_listed', 'genres_musical', 'genres_mystery', 'genres_no', 'genres_noir', 'genres_romance', 'genres_sci', 'genres_thriller', 'genres_war', 'genres_western']


In [16]:
category_dataframe = pd.DataFrame({"movieId": movies['movieId'],"title":movies['title'],"category":category})
pd.DataFrame(category_dataframe)

,movieId,title,category
0,1,Toy Story (1995),7
1,2,Jumanji (1995),8
2,3,Grumpier Old Men (1995),31
3,4,Waiting to Exhale (1995),16
4,5,Father of the Bride Part II (1995),2
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),35
9738,193583,No Game No Life: Zero (2017),17
9739,193585,Flint (2017),1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),43


In [17]:
same_cluster= category_dataframe[category_dataframe['category'] == 90]
pd.DataFrame(same_cluster.head(20))

,movieId,title,category


In [0]:
#Recommend 
Rec_clusterCB1=[]
for i in range(len(test_rating)):
  userid = int(test_rating.iloc[i]['user_id'])
  itemid = int(test_rating.iloc[i]['item_id'])
  same_mov=category_dataframe[(category_dataframe['movieId'] == itemid)]
  if len(same_mov)>0 :
    cluster=same_mov.iloc[0]['category']
    same_cluster= category_dataframe[category_dataframe['category'] == cluster]
    each_user = same_cluster.head(10)
    rec_movie = each_user['movieId'].unique()
    title_recmovie=each_user['title'].unique()
    Rec_clusterCB1.append([userid,itemid,rec_movie,title_recmovie])


In [19]:
Rec_CB1=pd.DataFrame(data=Rec_clusterCB1,columns=["userId","itemId","Rec_movieId","title"])
pd.DataFrame(Rec_CB1)

,userId,itemId,Rec_movieId,title
0,1,1,"[1, 13, 588, 709, 720, 1223, 1405, 1566, 2041,...","[Toy Story (1995), Balto (1995), Aladdin (1992..."
1,1,6,"[6, 23, 78, 165, 170, 185, 284, 288, 293, 330]","[Heat (1995), Assassins (1995), Crossing Guard..."
2,1,47,"[47, 50, 257, 335, 548, 695, 904, 911, 931, 933]","[Seven (a.k.a. Se7en) (1995), Usual Suspects, ..."
3,1,50,"[47, 50, 257, 335, 548, 695, 904, 911, 931, 933]","[Seven (a.k.a. Se7en) (1995), Usual Suspects, ..."
4,1,110,"[110, 151, 1208, 1224, 1233, 1262, 1408, 1927,...","[Braveheart (1995), Rob Roy (1995), Apocalypse..."
...,...,...,...,...
20163,610,160527,"[42, 384, 390, 493, 647, 694, 875, 1473, 2000,...","[Dead Presidents (1995), Bad Company (1995), F..."
20164,610,160571,"[177, 188, 220, 606, 735, 775, 841, 1105, 1128...","[Lord of Illusions (1995), Prophecy, The (1995..."
20165,610,163981,"[177, 188, 220, 606, 735, 775, 841, 1105, 1128...","[Lord of Illusions (1995), Prophecy, The (1995..."
20166,610,168248,"[6, 23, 78, 165, 170, 185, 284, 288, 293, 330]","[Heat (1995), Assassins (1995), Crossing Guard..."


# **Content Based with clustering by tag**

In [20]:
tags = pd.read_csv("/content/gdrive/My Drive/MovieLens/tags.csv") 
pd.DataFrame(tags)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [21]:
new_tags=[]
for i in range(len(tags)):
  user=tags.iloc[i]['userId']
  movie=tags.iloc[i]['movieId']
  tag=tags.iloc[i]['tag']
  if user not in new_tags and movie not in new_tags :
    same_index=tags[(tags.userId== user) & (tags.movieId == movie)]
    tagssss =same_index.tag.array
    s=" "
    teggs=s.join(tagssss)
    new_tags.append([user,movie,teggs])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [22]:
new_tags=np.array(new_tags)
new_tag = pd.DataFrame({"userId": new_tags[:,0],"movieId":new_tags[:,1],"tag":new_tags[:,2]})
new_tag=new_tag.drop_duplicates(keep='first')
pd.DataFrame(new_tag)

,userId,movieId,tag
0,2,60756,funny Highly quotable will ferrell
3,2,89774,Boxing story MMA Tom Hardy
6,2,106782,drugs Leonardo DiCaprio Martin Scorsese
9,7,48516,way too long
10,18,431,Al Pacino gangster mafia
...,...,...,...
3677,606,6107,World War II
3678,606,7382,for katie
3679,606,7936,austere
3680,610,3265,gun fu heroic bloodshed


In [0]:
cv_tag=CountVectorizer()
tag = cv_tag.fit_transform(new_tag["tag"])
tag_list = ["tag_"+ i for i in cv_tag.get_feature_names()]
cluster_data = np.hstack([tag.todense()*2])
mod = KMeans(n_clusters=50)
category = mod.fit_predict(cluster_data)

In [24]:
category_dataframeK2 = pd.DataFrame({"userId": new_tag['userId'],"movieId":new_tag['movieId'],"category":category})
pd.DataFrame(category_dataframeK2)

,userId,movieId,category
0,2,60756,0
3,2,89774,0
6,2,106782,0
9,7,48516,0
10,18,431,0
...,...,...,...
3677,606,6107,7
3678,606,7382,0
3679,606,7936,0
3680,610,3265,0


In [0]:
def get_title(rec_movie):
  title=[]
  for x in rec_movie :
    idmov=x 
    get_title=movies[movies['movieId'] == int(idmov)]
    rec=get_title.iloc[0]['title']
    title.append(rec)
  return title

In [0]:
npCategory=category_dataframeK2.to_numpy()
Rec_clusterCB2=[]
for i in range(len(test_rating)):
  userid = int(test_rating.iloc[i]['user_id'])
  itemid = int(test_rating.iloc[i]['item_id'])
  same_usmov=category_dataframeK2[(category_dataframeK2['movieId'] == str(itemid)) & (category_dataframeK2['userId']== str(userid))]
  if len(same_usmov) > 0:
    cluster=same_usmov.iloc[0]['category']
    same_cluster= category_dataframeK2[category_dataframeK2['category'] == cluster]
    each_user = same_cluster.head(10)
    rec_movie = each_user['movieId'].unique()
    title_recmovie=get_title(rec_movie)
    Rec_clusterCB2.append([userid,itemid,rec_movie,title_recmovie])

In [27]:
Rec_CB2=pd.DataFrame(data=Rec_clusterCB2,columns=["userId","itemId","Rec_movieId","title"])
pd.DataFrame(Rec_CB2)

,userId,itemId,Rec_movieId,title
0,2,89774,"[60756, 89774, 106782, 48516, 431, 1221, 5995,...","[Step Brothers (2008), Warrior (2011), Wolf of..."
1,18,144210,"[60756, 89774, 106782, 48516, 431, 1221, 5995,...","[Step Brothers (2008), Warrior (2011), Wolf of..."
2,21,119141,"[60756, 89774, 106782, 48516, 431, 1221, 5995,...","[Step Brothers (2008), Warrior (2011), Wolf of..."
3,62,2124,"[410, 2124, 27706, 38061, 60756, 61024, 71535,...","[Addams Family Values (1993), Addams Family, T..."
4,62,6564,"[60756, 89774, 106782, 48516, 431, 1221, 5995,...","[Step Brothers (2008), Warrior (2011), Wolf of..."
...,...,...,...,...
334,573,5064,"[5, 7, 1343, 1367, 2424, 2719, 4808, 6390, 694...","[Father of the Bride Part II (1995), Sabrina (..."
335,573,35836,"[60756, 89774, 106782, 48516, 431, 1221, 5995,...","[Step Brothers (2008), Warrior (2011), Wolf of..."
336,599,924,[924],[2001: A Space Odyssey (1968)]
337,606,6107,"[4223, 40278, 647, 920, 936, 1242, 1272, 2028,...","[Enemy at the Gates (2001), Jarhead (2005), Co..."


In [0]:
#berdasarkan data test di unionkan hasil dari colaborative, cb1 d
Final_Recommendation=[]
for i in range(len(test_rating)):
  userid=userid = int(test_rating.iloc[i]['user_id'])
  itemid = int(test_rating.iloc[i]['item_id'])
  A = Hasil_RecCF[Hasil_RecCF['user_id']== userid]
  B = Rec_CB1[(Rec_CB1['userId']== userid) & (Rec_CB1['itemId']== itemid)]
  C = Rec_CB2[(Rec_CB2['userId']==userid)  & (Rec_CB2['itemId']== itemid)]
  title_A=A.iloc[0]['recommendation']
  title_B = B.iloc[0]['title']
  if len(C)>0: 
    title_C=C.iloc[0]['title']
    each_user= list(set(title_A) | set(title_B) | set(title_C))
  else:
    each_user=list(set(title_A) | set(title_B))
  Final_Recommendation.append([userid,itemid,each_user])

In [31]:
Final_Rec=pd.DataFrame(data=Final_Recommendation,columns=["user_id","item_id","Recommendation"])
pd.DataFrame(Final_Rec)

,user_id,item_id,Recommendation
0,1,1,"[Dark Victory (1939), Time Bandits (1981), Win..."
1,1,6,"[Net, The (1995), Dark Victory (1939), Hackers..."
2,1,47,"[Dark Victory (1939), Rear Window (1954), Spel..."
3,1,50,"[Dark Victory (1939), Rear Window (1954), Spel..."
4,1,110,"[Dark Victory (1939), Last of the Mohicans, Th..."
...,...,...,...
20163,610,160527,"[12 Chairs (1971), Meet the Parents (2000), Se..."
20164,610,160571,"[12 Chairs (1971), Candyman: Farewell to the F..."
20165,610,163981,"[12 Chairs (1971), Candyman: Farewell to the F..."
20166,610,168248,"[12 Chairs (1971), Net, The (1995), Meet the P..."


In [0]:
Final_Rec.to_csv('Final_rec.csv', index=False)